Julia does not allow `^`, `'` and `*` in variable names. 

Thus, for variable names:

- superscript notation will elided
- $*$ will become `s`
- $'$ will become `p`
- $^-1$ will become `1`

Underscores for subscripts will be used

#### a) What is the basic feasible solution at this stage? What is the value of the objective?

#### 2017

In [28]:
cT = [ 20 10 15 0 0 0 0 ] # * -1 to minimize

A = [ 
    3 2 5 1 0 0 0;
    2 1 1 0 1 0 0;
    1 1 3 0 0 1 0;
    5 2 4 0 0 0 1
]

b = [
    55;
    26;
    30;
    57
]

basis = [1; 2; 4; 6]
non_basics = [3; 5; 7]

3-element Array{Int64,1}:
 3
 5
 7

#### 2016

In [29]:
cT = [ 2 1 2 0 0 0 ]

A = [
    4 3 8 1 0 0;
    4 1 12 0 1 0;
    4 -1 3 0 0 1
]

b = [
    12;
    8;
    8
]

basis = [1; 4; 6]
non_basics = [2; 3; 5]

3-element Array{Int64,1}:
 2
 3
 5

In [30]:
B = A[:, basis]

3×3 Array{Int64,2}:
 4  1  0
 4  0  0
 4  0  1

In [31]:
B1 = inv(B)

3×3 Array{Float64,2}:
 0.0   0.25  -0.0
 1.0  -1.0    0.0
 0.0  -1.0    1.0

In [32]:
# values of the variables of the basis
xB = B1 * b 

3-element Array{Float64,1}:
 2.0
 4.0
 0.0

In [33]:
# objective function coefficients of the basic variables
cBT = transpose(cT[basis])

1×3 RowVector{Int64,Array{Int64,1}}:
 2  0  0

In [34]:
# objective
zB = cBT * xB

4.0

#### b) What is the entering variable for the next step of the Revised Simplex Algorithm?

Determine the entering variable $x_{j^∗}$ with associated vector of the constraint matrix $p^{j^*}$. 

Compute the dual variables

In [35]:
yT = cBT * B1

1×3 RowVector{Float64,Array{Float64,1}}:
 0.0  0.5  0.0

Compute the "reduced cost" for all non-basic variables $$c^{'}_j = c_j - y^T p^j ~ \forall j \in \text{ non-basic}$$

In [36]:
cp = []

for j in non_basics
    pj = A[:, j]
    
    cp_j = cT[j] - (yT * pj)
    
    append!(cp, cp_j)
end

cp

3-element Array{Any,1}:
  0.5
 -4.0
 -0.5

Choose $j^∗$ such that $c^{'}_{j^*} > 0$ 

In [37]:
# c'_j*
(cp_js, i) = findmax(cp)

(0.5, 1)

In [38]:
js = non_basics[i]

# p^j*
pjs = A[:, js]

3-element Array{Int64,1}:
  3
  1
 -1

In [39]:
println("x", js, " should enter")

x2 should enter


#### c) What is the leaving variable?

Determine the leaving variable $x_r$ with associated vector $p^r$.

Compute the direction in which we are going to move $$a^{j^*} = B^{-1} p^{j^*}$$

for the $j^*$ we chose above.

As we increase $x_{j^*}$, then the other variables reduce by $a^{j^*}$.

In [40]:
ajs = B1 * pjs

3-element Array{Float64,1}:
  0.25
  2.0 
 -2.0 

$x_r$ is the basic variable which achieves the value $$\theta = \min_{k} \{  \frac{x^{B}_k}{\alpha^{j^*}_k} ~ \vert ~ \alpha^{j^*}_k > 0 \}$$

In [41]:
xs = [ (xB[k] / ajs[k], k) for k in 1:length(basis) if ajs[k] > 0 ]

2-element Array{Tuple{Float64,Int64},1}:
 (8.0, 1)
 (2.0, 2)

In [42]:
fst = x -> x[1]
(theta, k) = sort(xs, by=fst)[1]

(2.0, 2)

In [43]:
r = basis[k]
println("x", r, " should leave")

x4 should leave


Remove $x_r$ from the basis and add $x_{j^*}$

In [44]:
deleteat!(basis, basis .== r);
append!(basis, js)

deleteat!(non_basics, non_basics .== js);
append!(non_basics, r)

sort!(basis)

3-element Array{Int64,1}:
 1
 2
 6

In [45]:
sort!(non_basics)

3-element Array{Int64,1}:
 3
 4
 5

#### d) What is the new value of the objective? Verify that the new solution is optimal.

In [46]:
B = A[:, basis]

3×3 Array{Int64,2}:
 4   3  0
 4   1  0
 4  -1  1

In [47]:
B1 = inv(B)

3×3 Array{Float64,2}:
 -0.125   0.375  0.0
  0.5    -0.5    0.0
  1.0    -2.0    1.0

In [48]:
# values of the variables of the basis
xB = B1 * b 

3-element Array{Float64,1}:
 1.5
 2.0
 4.0

In [49]:
# objective function coefficients of the basic variables
cBT = transpose(cT[basis])

1×3 RowVector{Int64,Array{Int64,1}}:
 2  1  0

In [50]:
# objective 
zB = cBT * xB

5.0

In [51]:
yT = cBT * B1

1×3 RowVector{Float64,Array{Float64,1}}:
 0.25  0.25  0.0

In [52]:
cp = []

for j in non_basics
    pj = A[:, j]
    
    cp_j = cT[j] - (yT * pj)
    
    append!(cp, cp_j)
end

cp

3-element Array{Any,1}:
 -3.0 
 -0.25
 -0.25

If there are no positive reduced costs, then stop with the optimal solution

In [53]:
if all(x -> x <= 0, cp)
    print("Solution is optimal")
end

Solution is optimal

#### f) Assuming no other data changes, what value does the objective function coefficient of $𝑥_i$ have to reduce to so that $𝑥_i$ is zero in the optimal solution?

In [54]:
exam_2017 = 3
exam_2016 = 3

i = exam_2016

pi = A[:, i]

# c'_i = 0
# c'_i = c_i - (y^T * pJ)
# rearrange to
ci_ = yT * pi

5.0